In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split
import requests
from io import StringIO 
%matplotlib inline
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

In [6]:
ROOT = "/Users/kkalyan/github/airQo/"
train = pd.read_csv(ROOT+ "data/train-proc.csv")
test = pd.read_csv(ROOT+ "data/test-proc.csv")

In [41]:
test.shape

(5035, 770)

In [7]:
test.drop("target",axis  = 1,inplace  = True)

In [8]:
from sklearn.preprocessing import LabelEncoder
lab  = LabelEncoder()

In [9]:
lab.fit(train["location"])

LabelEncoder()

In [10]:
train.location  = lab.transform(train.location)
test.location  = lab.transform(test.location)

In [11]:
X=train.drop(["ID",'target'],axis =1)
y = train.target

In [13]:
from catboost import CatBoostRegressor

In [30]:
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=3)#15#5#10
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = CatBoostRegressor(n_estimators=7000, eval_metric='RMSE',learning_rate=0.1, random_seed= 1234, use_best_model=True )
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=100,)#erly100
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test.drop(["ID"],axis = 1))
    y_pred_totcb2.append(p2)
np.mean(errcb2)

0:	learn: 41.1995592	test: 41.1995592	test1: 41.7661222	best: 41.7661222 (0)	total: 87.3ms	remaining: 10m 11s
100:	learn: 26.4952257	test: 26.4952257	test1: 28.5392790	best: 28.5392790 (100)	total: 4.72s	remaining: 5m 22s
200:	learn: 22.9099361	test: 22.9099361	test1: 26.8325330	best: 26.8325330 (200)	total: 10.3s	remaining: 5m 47s
300:	learn: 20.4965104	test: 20.4965104	test1: 26.0705694	best: 26.0656560 (299)	total: 15s	remaining: 5m 32s
400:	learn: 18.5966018	test: 18.5966018	test1: 25.5993835	best: 25.5689894 (393)	total: 20.6s	remaining: 5m 39s
500:	learn: 17.0247232	test: 17.0247232	test1: 25.1813931	best: 25.1787196 (499)	total: 26s	remaining: 5m 37s
600:	learn: 15.6271025	test: 15.6271025	test1: 25.0234369	best: 25.0160412 (596)	total: 31s	remaining: 5m 29s
700:	learn: 14.4741363	test: 14.4741363	test1: 24.7543626	best: 24.7543626 (700)	total: 36s	remaining: 5m 23s
800:	learn: 13.3435636	test: 13.3435636	test1: 24.5998963	best: 24.5968133 (788)	total: 41.2s	remaining: 5m 19s
90

23.853337703439067

In [ ]:
from datetime import datetime
exp_time = str(datetime.now().replace(second=0, microsecond=0)).replace(" ", "_")
exp_name = "catboost-longer"
csv_file_name = "{}_{}".format(exp_time, exp_name)

In [ ]:
test_id  = test['ID']
d = {'ID': test_id, 'target': np.mean(y_pred_totcb2, 0)}
sub = pd.DataFrame(data=d)
sub = sub[['ID', 'target']]

In [ ]:
SUB = "/Users/kkalyan/github/airQo/submissions/"

In [ ]:
sub.to_csv(SUB+csv_file_name+'.csv', index=False)